In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [3]:
df = pd.read_excel('医院主数据20240520.xlsx',header=1)
df.head(5)

,医院编码,医院名称,别名,目标医院编码,目标医院名称,分级,分等,省,市,区,经营模式,类别,地址
0,110101001,北京市第六医院,NaN,110101001,北京市第六医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区交道口北二条36号
1,110101002,北京市公安医院,NaN,110101002,北京市公安医院,三级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区银闸胡同25号
2,110101003,北京市和平里医院,NaN,110101003,北京市和平里医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区和平里北街18号、东城区和平里西街19号楼一层12307号
3,110101004,北京市普仁医院,NaN,110101004,北京市普仁医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市崇文区崇外大街100号
4,110101005,中国医学科学院北京协和医院,NaN,110101005,中国医学科学院北京协和医院,三级,甲,北京市,北京城区,东城区,公立,综合,东城区东单帅府园1号(东院);西城区大木仓胡同41号(西院)


In [4]:
huanan_df = df.loc[df['省'].isin(['广东省', '广西壮族自治区', '海南省'])].reset_index(drop=True)

In [5]:
huanan_df

,医院编码,医院名称,别名,目标医院编码,目标医院名称,分级,分等,省,市,区,经营模式,类别,地址
0,440100696,广州中兴运动损伤专科医院,NaN,440100696,广州中兴运动损伤专科医院,二级,丙,广东省,广州市,白云区,民营,NaN,广州市白云区机场路松云街39号
1,440101001,广州开发区医院,NaN,440101001,广州开发区医院,三级,甲,广东省,广州市,黄埔区,公立,综合,广州市黄埔区开发区友谊路196号
2,440101002,广州市南沙中心医院,NaN,440101002,广州市南沙中心医院,三级,甲,广东省,广州市,南沙区,公立,NaN,广州市南沙开发区丰泽路105号
3,440101003,广州医科大学附属第一医院,NaN,440101003,广州医科大学附属第一医院,三级,甲,广东省,广州市,越秀区,公立,综合,广东省广州市越秀区沿江西路151号
4,440101004,南方医科大学南方医院,NaN,440101004,南方医科大学南方医院,三级,甲,广东省,广州市,白云区,公立,综合,广东省广州市广州大道北1838号
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019,469055491,陵水黎族自治县第二人民医院,NaN,469055491,陵水黎族自治县第二人民医院,二级,甲,海南省,陵水黎族自治县,陵水黎族自治县,公立,NaN,香水湾大道 16 号
6020,469068601,保亭县人民医院,NaN,469068601,保亭县人民医院,二级,甲,海南省,保亭黎族苗族自治县,保亭黎族苗族自治县,公立,NaN,海南省保亭黎族苗族自治县保城镇新兴东路58号
6021,469072568,万宁市人民医院,NaN,469072568,万宁市人民医院,二级,甲,海南省,万宁市,万宁市,公立,NaN,海南省万宁市环市三东路1号
6022,469085446,临高县中医院,NaN,469085446,临高县中医院,二级,甲,海南省,临高县,临高县,公立,NaN,临城镇临城解放路


In [6]:
hos_name = huanan_df['医院名称']

## 好大夫在线

In [ ]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}
hdf_error=[]
keshi_error=[]
response_error=[]
keshi_df=pd.DataFrame(columns=['医院名称','科室','来源'])
for name in hos_name:
    hdf_url = 'https://so.haodf.com/index/search?type=&kw='+name
    hdf_response = requests.get(hdf_url, headers=headers)
    if hdf_response.status_code == 200:
        hdf_soup = BeautifulSoup(hdf_response.content, 'html.parser')
        div_elements = hdf_soup.find('div', class_="js-search-list")
        try:
            exec("div_elements.find('a')['href']")
            keshi_url = div_elements.find('a')['href'].replace('.html','/keshi/list.html')
            keshi_response = requests.get(keshi_url, headers=headers)
            if keshi_response.status_code == 200:
                keshi_soup = BeautifulSoup(keshi_response.content, 'html.parser')
                keshi_elements = keshi_soup.find_all('div', class_="name-txt")
                texts = [element.get_text(strip=True) for element in keshi_elements]
                keshi_list = pd.DataFrame(texts,columns = ['科室'])
                for keshi in keshi_list['科室']:
                    keshi_df.loc[len(keshi_df)] = [name, keshi, '好大夫在线']
            else:
                keshi_error.append(name)
        except:
            hdf_error.append(name)
    else:
        response_error.append(name)

In [ ]:
len(keshi_error)

1359

In [ ]:
len(hdf_error)

770

In [ ]:
len(response_error)

0

In [ ]:
len(huanan_df)

6024

In [ ]:
len(hdf_error)+len(keshi_error)

2129

In [ ]:
keshi_df.to_excel('/Users/steven/Desktop/微创/华南医院数据.xlsx',index=False)

## 缺失数据查询

In [ ]:
missing_error = hdf_error+keshi_error

In [ ]:
missing_df = huanan_df[huanan_df['医院名称'].isin(missing_error)]
missing_df.groupby(['省','市']).size().reset_index(name='Count')

,省,市,Count
0,广东省,东莞市,87
1,广东省,中山市,92
2,广东省,云浮市,24
3,广东省,佛山市,288
4,广东省,惠州市,143
5,广东省,揭阳市,34
6,广东省,梅州市,27
7,广东省,汕头市,33
8,广东省,汕尾市,17
9,广东省,江门市,35
